import importlib
import tiktoken

print("tiktoken version:", importlib.metadata.version("tiktoken"))

tokenizer = tiktoken.get_encoding("gpt2")
text = (
    "Hello, do you like tea? <|endoftext|> In the sunlit terraces"
     "of someunknownPlace."
)

# region [token encoding]
integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})
# endregion
print (integers)

# region [token decoding]
strings = tokenizer.decode(integers)
# endregion
print (strings)





import os
import requests

os.makedirs("datas", exist_ok=True)
if not os.path.exists("datas/the-verdict.txt"):
    url = (
        "https://raw.githubusercontent.com/rasbt/"
        "LLMs-from-scratch/main/ch02/01_main-chapter-code/"
        "the-verdict.txt"
    )
    file_path = "datas/the-verdict.txt"

    response = requests.get(url, timeout=30)
    response.raise_for_status()
    with open(file_path, "wb") as f:
        f.write(response.content)
with open("datas/the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

enc_text = tokenizer.encode(raw_text)
print(len(enc_text))

enc_sample = enc_text[50:]

context_size=4

x=enc_sample[:context_size]
y=enc_sample[1:context_size+1]

print(f"x: {x}")
print(f"y:      {y}")

In [ ]:
import torch
from torch.utils.data import Dataset.DataLoader

class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
         """
        Args:
            txt (str): 학습할 전체 텍스트 데이터
            tokenizer: 텍스트를 토큰 ID로 변환해주는 토크나이저 (예: tiktoken)
            max_length (int): 모델이 한 번에 볼 수 있는 윈도우 크기 (입력 시퀀스 길이)
            stride (int): 윈도우를 이동시킬 간격 (데이터 중복 정도를 결정)
        """
        self.input_ids = []
        self.output_ids = []
        
        # 1. 전체 텍스트 토큰화
        # 텍스트를 정수 리스트(token_ids)로 변환합니다.
        # <|endoftext|> 같은 특수 토큰도 허용하여 인코딩합니다.
        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})
        
        # 데이터가 너무 짧으면 학습할 수 없으므로 최소 길이를 확인합니다.
        assert len(token_ids) > max_length, "토큰화된 입력의 개수는 적어도 max_length+1과 같아야 합니다."
        
        # 2. 슬라이딩 윈도우(Sliding Window)로 데이터 생성
        # 전체 토큰 리스트를 훑으며 max_length 길이만큼 잘라냅니다.
        
        for i in range(0, len(token_ids-max_length), stride):
            # 입력 청크: 현재 위치(i)부터 max_length만큼 가져옵니다.
            # region [input 청크 생성]
            input_chunk = token_ids[i:i+max_length]
            # endregion
            
            # 타겟 청크: 입력보다 1칸 뒤의 위치(i+1)부터 가져옵니다.
            # GPT는 '다음 단어'를 맞추는 모델이므로, 정답은 입력보다 한 칸씩 뒤로 밀려있어야 합니다.
            # region [target 청크 생성]
            target_chunk = token_ids[i+1; i+max_length+1]
            # endregion
            
            self.input_ids.append(torch.tensor(input_chunk))
            self.output_ids.append(torch.tensor(target_chunk))           
            
    def __len__(self):
        # 데이터셋의 총 샘플(청크) 개수를 반환합니다.
        return len(self.input_ids)

    def __getitem__(self, idx):
        # DataLoader가 데이터를 요청할 때 호출됩니다.
        # 해당 인덱스(idx)의 입력과 정답 쌍을 반환합니다.
        return self.input_ids[idx], self.target_ids[idx]
            
            

In [ ]:
##중요 : 아래 함수는 DataLoader를 생성하는 헬퍼 함수입니다.
import tiktoken

def create_dataloader_v1(txt, batch_size=4, max_length=256, 
                         stride=128, shuffle=True, drop_last=True, 
                         num_workers=0):
    
    # 토크나이저를 초기화합니다.
    tokenizer = tiktoken.get_encoding("gpt2")
    
    # 데이터셋을 만듭니다.
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)
    
    # 데이터 로더를 만듭니다.
    # region [DataLoader 생성]
    
    dataloader = DataLoader(
        dataset,
        batch_size= batch_size,
        shuffle=shuffle,
        drop_last = drop_last,
        num_workers=num_workers
    )
    # endregion
    
    return dataloader
    

    

In [ ]:
dataloader = create_dataloader_v1(
    raw_text, batch_size=1, max_length=4, stride=1, shuffle=False
)

data_iter = iter(dataloader)

first_batch = next(data_iter)
print(first_batch)

second_batch = next(data_iter)
print(second_batch)

input_ids = torch.tensor([2,3,5,1])

vocab_size = 6
output_dim = 3

torch.manual_seed(123)
# region [임베딩 레이어 생성]
embedding_layer = torch.nn.Embedding(vocab_size, output_dim)
# endregion

print(embedding_layer.weight)

# region [임베딩 레이어 적용]
token_embedding = embedding_layer(input_ids)
#endregion
print(token_embeddings)


In [ ]:
vocab_size = 50527
output_dim = 256

token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

max_length = 4
dataloader = create_dataloader_v1(
    raw_text, batch_size=8, max_length=max_length,
    stride=max_length, shuffle=False
)

data_iter = iter(dataloader)
inputs, targets = next(data_iter)
token_embeddings = token_embedding_layer(inputs)
print(token_embeddings.shape)

context_length = max_length
# region [포지셔널 임베딩 레이어 생성]
pos_embedding_layer = torch.nn.Embedding(context_length, outpu_dim)
# endregion

# region [포지셔널 임베딩 적용]
pos_embeddings = pos_embedding_layer(torch.arrange(max_length))
# endregion
print(pos_embeddings.shape)

# region [토큰 임베딩과 포지셔널 임베딩 합산]
input_embeddings = token_embeddings + pos_embeddings
# endregion
print(input_embeddings.shape

